# Analysis of Predictions Produced on Model Level with RNNModel Class by Darts

This script analyzes all pickle files in `./data/{approach}/{n_chunks}_chunks/{style}/`, starting with `confusion_matrix_models`, i.e. all model results. At the moment, the paths are adapted for local execution.

## Merge and Adjust Model Results

### Define Variables to Adjust

In [ ]:
# Define approach
approach = 'RNNModel'

# Define number of chunks (1000, 2000 or 15000)
n_chunks = 2000

# Define how many chunks were taken for prediction ('all' or '20_percent')
style = 'all'

# Defines if scaled values should be read and if yes, which scaled values
# '': read non-scaled results
# '_v1': read results produced with z-scaled series (same mean and same standard deviation)
# '_v2': read results produced with series scaled separately with several MinMaxScalers
scaling_version = '_v1'

### Extract All Generated Model Results

In [ ]:
import os
import pandas as pd
import pickle5 as pickle

# Define path to all model matrices produced by prediction
path_to_model_matrices = f'../../data/{approach}/{n_chunks}_chunks/{style}'

# Concat all found matrices into result matrix
result_matrix_models = pd.DataFrame(columns=['ID', 'PARAMETER', 'RUNTIME', 'MODEL', 'SCALED', 'LIBRARY', 'ENDOGENOUS',
                                             'EXOGENOUS', 'FIRST_FORECAST', 'ALARM_TYPE', 'FP', 'TP', 'FN', 'TN',
                                             'N_HIGH_ALARMS', 'N_LOW_ALARMS', 'N_CHUNKS', 'N_ITERATIONS'])

for file in os.listdir(path_to_model_matrices):
    if os.path.isfile(os.path.join(path_to_model_matrices, file)) and \
            file.startswith('confusion_matrix_models') and file.endswith(f'{scaling_version}.pickle'):

        # Read file
        current_matrix_f = open(f'{path_to_model_matrices}/{file}', 'rb')
        current_matrix = pickle.load(current_matrix_f)
        current_matrix_f.close()

        # Append current matrix to result matrix
        result_matrix_models = pd.concat([result_matrix_models, current_matrix])

# Align IDs
# Note: This is only needed for old 1000-20% runs were model numbers have ranged from '01' to '18'
if result_matrix_models['ID'].str.contains('18').any():

    # Remove additionally calculated rows
    result_matrix_models = result_matrix_models[~((result_matrix_models['ENDOGENOUS'] == 'MAX')
                                                  & (result_matrix_models['ALARM_TYPE'] == 'Low') |
                                                  (result_matrix_models['ENDOGENOUS'] == 'MIN')
                                                  & (result_matrix_models['ALARM_TYPE'] == 'High'))]

    # Update rows IDs of scaled results
    # RNN (min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('03', '02')
    # LSTM (median, max, min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('04', '03')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('05', '04')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('06', '04')
    # GRU (median, max, min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('07', '05')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('08', '06')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('09', '06')

    # Update rows IDs of non-scaled results
    # RNN (median, max, min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('10', '07')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('11', '08')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('12', '08')
    # LSTM (median, max, min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('13', '09')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('14', '10')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('15', '10')
    # GRU (median, max, min)
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('16', '11')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('17', '12')
    result_matrix_models['ID'] = result_matrix_models['ID'].str.replace('18', '12')

# Reset index
result_matrix_models.reset_index(inplace=True, drop=True)

# Add missing runtimes
# Note: This is only needed for old 2000-all runs where script was incomplete
if result_matrix_models['RUNTIME'].isnull().values.any():
    ids_with_missing_runtimes = result_matrix_models[result_matrix_models['RUNTIME'].isna()].ID

    for i, id_high in enumerate(ids_with_missing_runtimes):
        id_low = id_high.replace('H_', 'L_')
        new_runtime = result_matrix_models.loc[result_matrix_models['ID'] == id_low].RUNTIME
        result_matrix_models.at[ids_with_missing_runtimes.index[i], 'RUNTIME'] = new_runtime

### Add Metrics

See https://en.wikipedia.org/wiki/Sensitivity_and_specificity for more information.

In [ ]:
# Add confusion matrix ratios
result_matrix_models['FPR'] = result_matrix_models['FP'] / (result_matrix_models['FP'] + result_matrix_models['TN'])
result_matrix_models['TPR'] = result_matrix_models['TP'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['FNR'] = result_matrix_models['FN'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['TNR'] = result_matrix_models['TN'] / (result_matrix_models['FP'] + result_matrix_models['TN'])

# Add accuracy
result_matrix_models['ACC'] = (result_matrix_models['TP'] + result_matrix_models['TN']) / \
                              (result_matrix_models['TP'] + result_matrix_models['FN']
                               + result_matrix_models['FP'] + result_matrix_models['TN'])

# Add F1 score (harmonic mean of precision/PPV and sensitivity/TPR)
result_matrix_models['F1S'] = result_matrix_models['TP'] / \
                              (result_matrix_models['TP'] +
                               0.5 * (result_matrix_models['FP'] + result_matrix_models['FN']))

# Add threat score
result_matrix_models['TS'] = result_matrix_models['TP'] / (result_matrix_models['TP'] + result_matrix_models['FN'] + result_matrix_models['FP'])

# Add Matthews correlation coefficient
result_matrix_models['MCC_divident'] = result_matrix_models['TP'] * result_matrix_models['TN'] - \
                                       result_matrix_models['FP'] * result_matrix_models['FN']
result_matrix_models['MCC_divisor'] = ((result_matrix_models['TP'] + result_matrix_models['FP']) *
                                              (result_matrix_models['TP'] + result_matrix_models['FN']) *
                                              (result_matrix_models['TN'] + result_matrix_models['FP']) *
                                              (result_matrix_models['TN'] + result_matrix_models['FN']))**(1/2)
result_matrix_models['MCC'] = result_matrix_models['MCC_divident'] / result_matrix_models['MCC_divisor']

# Add weighted score from https://physionet.org/content/challenge-2015/1.0.0/
# Original: (TP + TN) / (TP + TN + FP + 5*FN)
# Adapted: (TP) / (TP + FN + 5*FP)
result_matrix_models['WEIGHTED_SCORE_AA'] = result_matrix_models['TP'] / \
                                            (result_matrix_models['TP'] + result_matrix_models['FN'] +
                                             (5 * result_matrix_models['FP']))

# Round all floats to 4 decimal places
# Note: round() does not work for floats with many decimal places
decimals = 4
for col in ['FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S', 'TS', 'MCC', 'WEIGHTED_SCORE_AA']:
    result_matrix_models[col] = result_matrix_models[col].apply(lambda x: round(x, decimals))

# Move cols to end for similarity with ARIMA results and drop columns (alarm counts and MCC helper columns)
result_matrix_models = result_matrix_models[['ID', 'PARAMETER', 'RUNTIME', 'MODEL', 'SCALED', 'LIBRARY', 'ENDOGENOUS',
                                             'EXOGENOUS', 'FIRST_FORECAST', 'ALARM_TYPE', 'TP', 'FN', 'FP', 'TN',
                                             'FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S', 'MCC', 'TS', 'WEIGHTED_SCORE_AA',
                                             'N_CHUNKS', 'N_ITERATIONS']]

### Add Temporary Missing Rows

In [ ]:
import numpy as np

# Get missing IDs
if scaling_version == '':
    range_min = 6
    range_max = 12
    id_suffix = ''
else:
    range_min = 0
    range_max = 6
    id_suffix = scaling_version

id_prefixes = [param + '_R_' + str(nr + 1) if nr >= 9 else param + '_R_0' + str(nr + 1)
               for nr in list(range(12))[range_min:range_max] for param in ['BP', 'HR', 'O2']]

all_ids = [id_prefix + '_H' + id_suffix for id_prefix in id_prefixes] + [id_prefix + '_L' + id_suffix for id_prefix in id_prefixes]

missing_ids = list(set(all_ids).difference(set(pd.unique(result_matrix_models.ID))))
print(f'Missing IDs: {missing_ids}')

if len(missing_ids) > 0:

    # Add missing rows with columns that are not row-specific
    missing_rows = pd.DataFrame(data={'ID': list(missing_ids),
                                      'LIBRARY' : ['darts'] * len(missing_ids),
                                      'FIRST_FORECAST' : [12] * len(missing_ids)})

    # Add row-specific columns
    missing_rows['PARAMETER'] = missing_rows['ID'].str[:2]
    missing_rows['ALARM_TYPE'] = ['Low' if model_id[-1] == 'L' else 'High' for model_id in missing_rows['ID'] ]

    scaled_model_numbers = ['0' + str(nr) for nr in list(range(7))][1:]
    missing_rows['SCALED'] = [True if model_id.split('_')[2] in scaled_model_numbers else False for model_id in missing_rows['ID']]

    missing_rows['MODEL'] = ['RNN' if model_id.split('_')[2] in ['01', '02', '07', '08']
                             else 'LSTM' if model_id.split('_')[2] in ['03', '04', '09', '10']
                             else 'GRU' for model_id in missing_rows['ID']]

    missing_rows['ENDOGENOUS'] = ['Median' if model_id.split('_')[2] in ['01', '07', '03', '09', '05', '11']
                                  else 'Max' if model_id[-1] == 'H'
                                  else 'Min' for model_id in missing_rows['ID']]

    missing_rows['EXOGENOUS'] = [np.nan if model_id.split('_')[2] in ['01', '07', '03', '09', '05', '11']
                                 else 'Median' for model_id in missing_rows['ID']]

    # Add missing rows (and fill missing columns with NaN by default)
    result_matrix_models = pd.concat([result_matrix_models, missing_rows], axis=0, ignore_index=True)

### Finalize and Save as Parquet File

In [ ]:
from IPython.display import display

# Adjust data types
result_matrix_models['SCALED'] = result_matrix_models['SCALED'].astype(bool)
result_matrix_models['FIRST_FORECAST'] = result_matrix_models['FIRST_FORECAST'].astype(int)

if len(missing_ids) == 0:
    result_matrix_models['RUNTIME'] = result_matrix_models['RUNTIME'].astype(int)
    result_matrix_models['TP'] = result_matrix_models['TP'].astype(int)
    result_matrix_models['TN'] = result_matrix_models['TN'].astype(int)
    result_matrix_models['FP'] = result_matrix_models['FP'].astype(int)
    result_matrix_models['FN'] = result_matrix_models['FN'].astype(int)

# Sort result matrix for better readability
result_matrix_models.sort_values(by=['ID'], inplace=True)

# Reset index
result_matrix_models.reset_index(inplace=True, drop=True)

# Show result matrix per parameter
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'BP'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'HR'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'O2'])

# Save as parquet file
result_matrix_models.to_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}'
                                f'{scaling_version}.parquet', engine='pyarrow')

### Combine Different Result Matrices

In [ ]:
# Read non-scaled results
results_normal = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}'
                                 f'.parquet', engine='pyarrow')

# Remove wrong 01 to 06 entries
results_normal = results_normal[results_normal['ID'].str.contains('07|08|09|10|11|12')]


# Read z-scaled results
results_z_scaled = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                   f'{style}_v1.parquet', engine='pyarrow')

# Read min-max-scaled results
results_min_max_scaled = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                         f'{style}_v2.parquet', engine='pyarrow')

# Remove IDs 07 to 12
results_z_scaled = results_z_scaled[results_z_scaled['ID'].str.contains('01|02|03|04|05|06')]
results_min_max_scaled = results_min_max_scaled[results_min_max_scaled['ID'].str.contains('01|02|03|04|05|06')]


# Concat both result matrices
rnn_model_results = pd.concat([results_normal, results_z_scaled, results_min_max_scaled], axis=0, ignore_index=True)

# Sort result matrix for better readability
rnn_model_results.sort_values(by=['ID'], inplace=True)

# Reset index
rnn_model_results.reset_index(inplace=True, drop=True)

# Show result matrix per parameter
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'BP'])
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'HR'])
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'O2'])

# Save as parquet file
rnn_model_results.to_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}_'
                             f'final.parquet', engine='pyarrow')

## Visualization of Model Level Results

### Setup Variables for All Model Level Plots

In [ ]:
approach = 'RNNModel'
n_chunks = 2000
style = 'all'

# Suffix can be '', '_v1, '_v2' or '_final'
suffix = '_final'

# Avoid error because of non-found values
available_parameters = pd.unique(result_matrix_models.PARAMETER)

# Only plot columns for available parameters
n_cols = len(available_parameters)

result_matrix_models = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                       f'{style}{suffix}.parquet', engine='pyarrow')

### Plot Accuracy, TPR, FNR, and TNR

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=4,
    ncols=n_cols,
    figsize=(15, 13),
    dpi=72
    )

#plt.suptitle(f'Accuracy, TPR, FNR, and TNR of {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Define y-limits
acc_ylimits = [0, result_matrix_models.ACC.max(skipna=True)]
tpr_ylimits = [0, result_matrix_models.TPR.max(skipna=True)]
fnr_ylimits = [0, result_matrix_models.FNR.max(skipna=True)]
tnr_ylimits = [0, result_matrix_models.TNR.max(skipna=True)]

# Actual plots
for i, parameter in enumerate(available_parameters):

    sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='ACC',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(str(parameter), fontweight='bold', color='black', fontsize=20)
    axs[0, i].set_ylim(acc_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(tpr_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax=axs[2, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[2, i].set_ylim(fnr_ylimits)
    axs[2, i].set_xticklabels(axs[2, i].get_xticklabels(), rotation=90)
    axs[2, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax = axs[3, i],
        data = plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[3, i].set_ylim(tnr_ylimits)
    axs[3, i].set_xticklabels(axs[3, i].get_xticklabels(), rotation=90)
    axs[3, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_tpr_fnr_tnr_acc.png', dpi=72)

### Plot False Positive Ratio and F1 Score

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=2,
    ncols=n_cols,
    figsize=(15, 7),
    dpi=72
    )

#yplt.suptitle(f'FPR and F1S of {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Define y-limits
fpr_ylimits = [0, result_matrix_models.FPR.max(skipna=True)]
f1s_ylimits = [0, result_matrix_models.F1S.max(skipna=True)]

# Actual plot
for i, parameter in enumerate(available_parameters):

    print(f'\n##### {parameter} #####')

    g_fpr = sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(str(parameter), fontweight='bold', color='black', fontsize=14)
    axs[0, i].set_ylim(fpr_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best FPR
    best_fpr_low = plotdata[plotdata.FPR == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].FPR.min()]['FPR'].unique()
    print(f'Best low FPR: {best_fpr_low}')
    best_fpr_high = plotdata[plotdata.FPR == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].FPR.min()]['FPR'].unique()
    print(f'Best high FPR: {best_fpr_high}')

    # Add red rectangle around models with best F1 score
    for bar in g_fpr.patches:
        if bar.get_height() == best_fpr_low or bar.get_height() == best_fpr_high:
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

    g_f1s = sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='F1S',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(f1s_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best F1 scores
    best_f1s_low = plotdata[plotdata.F1S == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].F1S.max()]['F1S'].unique()
    print(f'Best low F1S: {best_f1s_low}')
    best_f1s_high = plotdata[plotdata.F1S == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].F1S.max()]['F1S'].unique()
    print(f'Best high F1S: {best_f1s_high}')

    # Add red rectangle around models with best F1 score
    for bar in g_f1s.patches:
        if bar.get_height() == best_f1s_low or bar.get_height() == best_f1s_high:
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_fpr_f1s.png', dpi=72)

### Plot TS, Weighted Score & MCC

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

sns.set_style('whitegrid')

fig, axs = plt.subplots(
    nrows=3,
    ncols=n_cols,
    figsize=(20, 12),
    dpi=72
    )
fig.subplots_adjust(hspace=0.6, wspace=0.4)

ts_ylimits = [0, result_matrix_models.TS.max(skipna=True)]
weighted_aa_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True)]
mcc_ylimits = [0, result_matrix_models.MCC.max(skipna=True)]

for i, parameter in enumerate(available_parameters):

    print(f'\n##### {parameter} #####')

    g_ts = sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TS',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(str(parameter), fontweight='bold', color='black', fontsize=14)
    axs[0, i].set_ylim(ts_ylimits)
    axs[0, i].tick_params(axis='x', rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best TS
    best_ts_low = plotdata[plotdata.TS == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].TS.max()]['TS'].unique()
    print(f'Best low TS: {best_ts_low}')
    best_ts_high = plotdata[plotdata.TS == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].TS.max()]['TS'].unique()
    print(f'Best high TS: {best_ts_high}')

    # Add red rectangle around models with best TS
    for bar in g_ts.patches:
        if bar.get_height() == best_ts_low or bar.get_height() == best_ts_high :
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

    g_ws_aa = sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(weighted_aa_score_ylimits)
    axs[1, i].tick_params(axis='x', rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best WS_AA
    best_weighted_aa_score_low = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].WEIGHTED_SCORE_AA.max()]['WEIGHTED_SCORE_AA'].unique()
    print(f'Best low WS_AA: {best_weighted_aa_score_low}')
    best_weighted_aa_score_high = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].WEIGHTED_SCORE_AA.max()]['WEIGHTED_SCORE_AA'].unique()
    print(f'Best high WS_AA: {best_weighted_aa_score_high}')

    # Add red rectangle around models with best WS_AA
    for bar in g_ws_aa.patches:
        if bar.get_height() == best_weighted_aa_score_low or bar.get_height() == best_weighted_aa_score_high :
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

    g_mcc = sns.barplot(
        ax=axs[2, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='MCC',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[2, i].set_ylim(mcc_ylimits)
    axs[2, i].tick_params(axis='x', rotation=90)
    axs[2, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best MCC
    best_mcc_low = plotdata[plotdata.MCC == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].MCC.max()]['MCC'].unique()
    print(f'Best low MCC: {best_mcc_low}')
    best_mcc_high = plotdata[plotdata.MCC == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].MCC.max()]['MCC'].unique()
    print(f'Best high MCC: {best_mcc_high}')

    # Add red rectangle around models with best MCC
    for bar in g_mcc.patches:
        if bar.get_height() == best_mcc_low or bar.get_height() == best_mcc_high :
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

# Save figure
plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_ts_ws_mcc.png', dpi=72, bbox_inches='tight')
plt.show(fig)

In [ ]:
# TODO: Add plots comparing model types and scaling influence

### Plot Runtimes

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plotdata = result_matrix_models.replace(['_H', '_L'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=1,
    ncols=n_cols,
    figsize=(15, 7),
    dpi=72
    )

# Add main title
plt.suptitle(f'Runtime of Predictions with {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Add actual plot and adjust texts
for i, parameter in enumerate(available_parameters):

    sns.barplot(
        ax=axs[i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='RUNTIME',
        color=sns.color_palette('colorblind')[0],
        ci=None)
    axs[i].set_title(str(parameter), fontweight='bold', color='black', fontsize=14)
    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=90)
    axs[i].set_yticklabels(pd.to_datetime(axs[i].get_yticks(), unit='s').strftime('%d Days %H:%M:%S'))

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_runtimes.png', dpi=72)